In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

from sklearn.neighbors import NearestNeighbors

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
books = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/data/books.csv")
books.to_csv(r"/content/drive/MyDrive/Colab Notebooks/data/books.csv")
books.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,0,0,0,0,0,0,0,0,0,0,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,1,1,1,1,1,1,1,1,1,1,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,2,2,2,2,2,2,2,2,2,2,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,3,3,3,3,3,3,3,3,3,3,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,4,4,4,4,4,4,4,4,4,4,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [4]:
ratings = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/data/ratings.csv")
ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


FILTER POPULAR BOOKS

In [5]:
print (books['books_count'].describe())

count   10000.000
mean       75.713
std       170.471
min         1.000
25%        23.000
50%        40.000
75%        67.000
max      3455.000
Name: books_count, dtype: float64


In [6]:
print (ratings['rating'].describe())

count   981756.000
mean         3.857
std          0.984
min          1.000
25%          3.000
50%          4.000
75%          5.000
max          5.000
Name: rating, dtype: float64


In [7]:
popularity_threshold = 40
rating_popular_book = books.loc[books['books_count'] >= popularity_threshold]
rating_popular_book.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,0,0,0,0,0,0,0,0,0,0,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,1,1,1,1,1,1,1,1,1,1,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,2,2,2,2,2,2,2,2,2,2,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,3,3,3,3,3,3,3,3,3,3,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,4,4,4,4,4,4,4,4,4,4,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [8]:
rating_popular_book.shape

(5039, 46)

In [9]:
rating_popular_book = rating_popular_book.dropna()
ratings = ratings.sort_values("user_id")
ratings.drop_duplicates(subset=["user_id","book_id"], keep='first', inplace=True) 
rating_popular_book.drop_duplicates(subset='original_title', keep='first', inplace=True)

In [10]:
#display all the language
#print(books.language_code.to_string(index=False))
rating_popular_book['language_code'].value_counts()

eng      3163
en-US     962
en-GB     114
en-CA      22
spa        18
fre        18
ger        11
por         5
pol         5
dan         3
jpn         3
ita         2
ind         2
ara         2
per         1
nor         1
vie         1
nl          1
swe         1
Name: language_code, dtype: int64

In [11]:
#remove other language other than eng
#df.drop(df[(df['Unit_Price'] >400) & (df['Unit_Price'] < 600)].index, inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "fre")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "spa")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "ger")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "ind")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "jpn")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "ara")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "por")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "pol")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "dan")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "ita")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "per")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "tur")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "rum")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "fil")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "nor")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "swe")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "nl")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "vie")],axis=0,inplace=True)
rating_popular_book.drop(rating_popular_book.index[(rating_popular_book["language_code"] == "rus")],axis=0,inplace=True)
rating_popular_book['language_code'].value_counts()

eng      3163
en-US     962
en-GB     114
en-CA      22
Name: language_code, dtype: int64

In [12]:
#replace en-US, etc to eng
#df['column name'] = df['column name'].replace(['old value'],'new value')
rating_popular_book['language_code'] = rating_popular_book['language_code'].replace(['en-US','eng'])
rating_popular_book['language_code'] = rating_popular_book['language_code'].replace(['en-GB','eng'])
rating_popular_book['language_code'] = rating_popular_book['language_code'].replace(['en-CA','eng'])
rating_popular_book['language_code'] = rating_popular_book['language_code'].replace(['en','eng'])
rating_popular_book['language_code'].value_counts()

eng    4261
Name: language_code, dtype: int64

In [13]:
#Replaced the clean dataset into the old one
rating_popular_book.to_csv(r"/content/drive/MyDrive/Colab Notebooks/data/popular_books.csv")

rating_popular_books = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/data/popular_bookss.csv")
rating_popular_books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,genre
0,1,2767052,2767052,2792775,272,439023483,9780439023480,Suzanne Collins,2008,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Young Adult
1,2,3,3,4640799,491,439554934,9780439554930,"J.K. Rowling, Mary GrandPré",1997,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,Young Adult
2,3,41865,41865,3212258,226,316015849,9780316015840,Stephenie Meyer,2005,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,Young Adult
3,4,2657,2657,3275794,487,61120081,9780061120080,Harper Lee,1960,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,Classics
4,5,4671,4671,245494,1356,743273567,9780743273560,F. Scott Fitzgerald,1925,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,Classics


In [14]:
#display all the genres
#print(books.language_code.to_string(index=False))
rating_popular_books['genre'].value_counts()

Young Adult        544
Non-fiction        536
Classics           514
Mystery            479
Romance            443
Historical         347
Fantasy            345
Fiction            253
Science Fiction    202
Thriller           195
Horror             169
Children           154
Crime               40
Graphic Novel       34
Christian            4
Box set              1
Myths                1
Name: genre, dtype: int64

In [15]:
#keep only fiction books
rating_popular_books.drop(rating_popular_books.index[(rating_popular_books["genre"] == "Non-fiction")],axis=0,inplace=True)
rating_popular_books.drop(rating_popular_books.index[(rating_popular_books["genre"] == "Children")],axis=0,inplace=True)
rating_popular_books.drop(rating_popular_books.index[(rating_popular_books["genre"] == "Graphic Novel")],axis=0,inplace=True)
rating_popular_books.drop(rating_popular_books.index[(rating_popular_books["genre"] == "Christian")],axis=0,inplace=True)
rating_popular_books.drop(rating_popular_books.index[(rating_popular_books["genre"] == "Myths")],axis=0,inplace=True)
rating_popular_books.drop(rating_popular_books.index[(rating_popular_books["genre"] == "Box set")],axis=0,inplace=True)
rating_popular_books['genre'].value_counts()

Young Adult        544
Classics           514
Mystery            479
Romance            443
Historical         347
Fantasy            345
Fiction            253
Science Fiction    202
Thriller           195
Horror             169
Crime               40
Name: genre, dtype: int64

In [16]:
merged_df = pd.merge(rating_popular_books, ratings, how='left', left_on=['id'], right_on=['book_id'])
df = merged_df[['id','original_title', 'user_id', 'rating', 'books_count', 'language_code', 'genre']]

df = df.rename(columns = {'id':'book_id'})
df.head(200)

,book_id,original_title,user_id,rating,books_count,language_code,genre
0,1,The Hunger Games,314,5,272,eng,Young Adult
1,1,The Hunger Games,439,3,272,eng,Young Adult
2,1,The Hunger Games,588,5,272,eng,Young Adult
3,1,The Hunger Games,1169,4,272,eng,Young Adult
4,1,The Hunger Games,1185,4,272,eng,Young Adult
...,...,...,...,...,...,...,...
195,2,Harry Potter and the Philosopher's Stone,51166,5,491,eng,Young Adult
196,2,Harry Potter and the Philosopher's Stone,51460,4,491,eng,Young Adult
197,2,Harry Potter and the Philosopher's Stone,51838,4,491,eng,Young Adult
198,2,Harry Potter and the Philosopher's Stone,52036,3,491,eng,Young Adult


In [17]:
df.describe()

,book_id,user_id,rating,books_count
count,350955.000,350955.000,350955.000,350955.000
mean,3675.273,24323.231,3.788,133.560
std,2761.310,15230.858,0.982,240.594
min,1.000,1.000,1.000,40.000
25%,1285.000,11055.000,3.000,52.000
50%,3106.000,22786.000,4.000,68.000
75%,5678.000,37041.000,5.000,103.000
max,9998.000,53424.000,5.000,3455.000


MAKING RECOMMENDATION

ALT CODING (SET RANDOM)

```
df_data_pivot = df.pivot(index = 'original_title', columns = 'user_id', values = 'rating').fillna(0)
df_data_matrix = csr_matrix(df_data_pivot.values)

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(df_data_pivot)

query_index = np.random.choice(df.shape[0])
print (query_index)
distances, indices = model_knn.kneighbors(df_data_pivot.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)

df_data_pivot.index[query_index]

for i in range(0, len(distances.flatten())):
    if i == 0:
        print ('Recommendations for {0}:\n'.format(df_data_pivot.index[query_index]))
    else:
        print ('{0}: {1}, with distance of {2}:'.format(i, df_data_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

```



In [18]:
ratings_df = df.pivot_table(index='book_id',columns='user_id',values='rating').fillna(0)

pd.set_option('display.max_columns', 100)
ratings_df.head()

user_id,1,4,7,8,9,10,11,13,16,18,19,20,22,23,24,25,26,28,29,30,31,32,33,34,35,36,39,40,41,42,44,46,47,48,49,50,51,52,54,55,57,58,59,60,61,62,63,65,66,72,...,53337,53338,53339,53341,53344,53345,53347,53348,53350,53351,53352,53354,53355,53358,53360,53364,53366,53367,53369,53371,53372,53373,53374,53378,53379,53380,53381,53384,53387,53388,53389,53390,53393,53396,53397,53398,53399,53400,53401,53403,53409,53411,53412,53413,53414,53419,53421,53422,53423,53424
book_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [19]:
ratings_df.shape

(3531, 31837)

In [20]:
ratings_matrix = csr_matrix(ratings_df.values)

model_knn = NearestNeighbors(metric='cosine', algorithm = 'brute')
model_knn.fit(ratings_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [21]:
!pip install recmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit-surprise-1.1.1.tar.gz (11.8 MB)
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633729 sha256=adfb5d5f48c37ec6ecb625c1aee7c014e5d0c54081e857366daa6c7fd673490f
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [23]:
import matplotlib.pyplot as plt
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
import recmetrics

In [24]:
def get_book_id(book_title):
    target_df = df.loc[df['original_title'] == book_title]
    return target_df['book_id'].iloc[0]

id_TheHungerGames = get_book_id('The Hunger Games')
print(id_TheHungerGames)

def get_title(book_id):
    target_df = df.loc[df['book_id'] == book_id]
    return target_df['original_title'].iloc[0]

print(get_title(1))

def get_recomm(book_title, num_neighbors=10, display=False): 
    book_ids = []

    query_index = get_book_id(book_title) - 1
    
    if num_neighbors > 0:
        distances, indices = model_knn.kneighbors(ratings_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = num_neighbors + 1)
    else:
        distances, indices = model_knn.kneighbors(ratings_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 10 + 1)
    
    for i in range(0, len(distances.flatten())):

        if display is True:
            if i == 0:
                print('Recommendations for ', book_title, '\n')
            else:    
                print('{0}\t Book ID: {1}\t  Distance: {2}:\n'.format(i, ratings_df.index[indices.flatten()[i]], distances.flatten()[i]))
        
        book_ids.append(ratings_df.index[indices.flatten()[i]])

    return book_ids

1
The Hunger Games


In [25]:
# Top 10 recommendations for The Hunger Games
recommendations_for_TheHungerGames = get_recomm('The Hunger Games', num_neighbors=10, display=True)

for b in recommendations_for_TheHungerGames[1:]:
    print('id:', b, '\t\tBook: ', get_title(b))


reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'book_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()
algo.fit(trainset)

test = algo.test(testset)
test = pd.DataFrame(test)
test.drop("details", inplace=True, axis=1)
test.columns = ['user_id', 'book_id', 'actual', 'cf_predictions']
test.head()
print("MSE: ", recmetrics.mse(test.actual, test.cf_predictions))
print("RMSE: ", recmetrics.rmse(test.actual, test.cf_predictions))

Recommendations for  The Hunger Games 

1	 Book ID: 17	  Distance: 0.4053256862294832:

2	 Book ID: 31	  Distance: 0.42674642418525066:

3	 Book ID: 2	  Distance: 0.4444738573252147:

4	 Book ID: 20	  Distance: 0.4523031603835689:

5	 Book ID: 3	  Distance: 0.49084774300774026:

6	 Book ID: 5	  Distance: 0.5197672572777041:

7	 Book ID: 16	  Distance: 0.5229117539369219:

8	 Book ID: 9	  Distance: 0.5271915645842881:

9	 Book ID: 37	  Distance: 0.5278079122055651:

10	 Book ID: 36	  Distance: 0.5335204712425707:

id: 17 		Book:  Catching Fire
id: 31 		Book:  The Help
id: 2 		Book:  Harry Potter and the Philosopher's Stone
id: 20 		Book:  Mockingjay
id: 3 		Book:  Twilight
id: 5 		Book:  The Great Gatsby
id: 16 		Book:  Män som hatar kvinnor
id: 9 		Book:  Angels & Demons
id: 37 		Book:  The Lion, the Witch and the Wardrobe
id: 36 		Book:  The Giver
MSE:  0.726855826481271
RMSE:  0.85255840062794


In [26]:
# Top 10 recommendations for Pride and Prejudice
recommendations_for_PrideandPrejudice = get_recomm('Pride and Prejudice', num_neighbors=10, display=True)

for b in recommendations_for_PrideandPrejudice[1:]:
    print('id:', b, '\t\tBook: ', get_title(b))

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'book_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()
algo.fit(trainset)

test = algo.test(testset)
test = pd.DataFrame(test)
test.drop("details", inplace=True, axis=1)
test.columns = ['user_id', 'book_id', 'actual', 'cf_predictions']
test.head()
print("MSE: ", recmetrics.mse(test.actual, test.cf_predictions))
print("RMSE: ", recmetrics.rmse(test.actual, test.cf_predictions))

Recommendations for  Pride and Prejudice 

1	 Book ID: 43	  Distance: 0.22787681057242637:

2	 Book ID: 13	  Distance: 0.36533218160906544:

3	 Book ID: 76	  Distance: 0.3858151683688861:

4	 Book ID: 33	  Distance: 0.39752596233629034:

5	 Book ID: 8	  Distance: 0.40703356373437594:

6	 Book ID: 4	  Distance: 0.4251608152166305:

7	 Book ID: 63	  Distance: 0.4303767548620868:

8	 Book ID: 5	  Distance: 0.44601375834249446:

9	 Book ID: 42	  Distance: 0.45122474447064265:

10	 Book ID: 16	  Distance: 0.4518157383086868:

id: 43 		Book:  Jane Eyre
id: 13 		Book:  Nineteen Eighty-Four
id: 76 		Book:  Sense and Sensibility
id: 33 		Book:  Memoirs of a Geisha
id: 8 		Book:  The Catcher in the Rye
id: 4 		Book:  To Kill a Mockingbird
id: 63 		Book:  Wuthering Heights
id: 5 		Book:  The Great Gatsby
id: 42 		Book:  Little Women
id: 16 		Book:  Män som hatar kvinnor
MSE:  0.7285752665434129
RMSE:  0.853566205131982


In [27]:
# Top 10 recommendations for The Hobbit or There and Back Again
recommendations_for_TheHobbitorThereandBackAgain = get_recomm('The Hobbit or There and Back Again', num_neighbors=10, display=True)

for b in recommendations_for_TheHobbitorThereandBackAgain[1:]:
    print('id:', b, '\t\tBook: ', get_title(b))

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'book_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()
algo.fit(trainset)

test = algo.test(testset)
test = pd.DataFrame(test)
test.drop("details", inplace=True, axis=1)
test.columns = ['user_id', 'book_id', 'actual', 'cf_predictions']
test.head()
print("MSE: ", recmetrics.mse(test.actual, test.cf_predictions))
print("RMSE: ", recmetrics.rmse(test.actual, test.cf_predictions))

Recommendations for  The Hobbit or There and Back Again 

1	 Book ID: 19	  Distance: 0.34216799209825444:

2	 Book ID: 2	  Distance: 0.42733371842918766:

3	 Book ID: 5	  Distance: 0.45937954366767486:

4	 Book ID: 37	  Distance: 0.46623080883255696:

5	 Book ID: 4	  Distance: 0.4825436237264442:

6	 Book ID: 58	  Distance: 0.4925815546644047:

7	 Book ID: 32	  Distance: 0.5007503857150721:

8	 Book ID: 9	  Distance: 0.5018479197268035:

9	 Book ID: 116	  Distance: 0.5032333620709142:

10	 Book ID: 28	  Distance: 0.5040779090701413:

id: 19 		Book:  The Fellowship of the Ring
id: 2 		Book:  Harry Potter and the Philosopher's Stone
id: 5 		Book:  The Great Gatsby
id: 37 		Book:  The Lion, the Witch and the Wardrobe
id: 4 		Book:  To Kill a Mockingbird
id: 58 		Book:  The Adventures of Huckleberry Finn
id: 32 		Book:  Of Mice and Men
id: 9 		Book:  Angels & Demons
id: 116 		Book:  The Adventures of Tom Sawyer
id: 28 		Book:  Lord of the Flies
MSE:  0.725364651800592
RMSE:  0.851683422288

In [28]:
# Top 10 recommendations for American Gods
recommendations_for_AmericanGods = get_recomm('American Gods', num_neighbors=10, display=True)

for b in recommendations_for_AmericanGods[1:]:
    print('id:', b, '\t\tBook: ', get_title(b))

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'book_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()
algo.fit(trainset)

test = algo.test(testset)
test = pd.DataFrame(test)
test.drop("details", inplace=True, axis=1)
test.columns = ['user_id', 'book_id', 'actual', 'cf_predictions']
test.head()
print("MSE: ", recmetrics.mse(test.actual, test.cf_predictions))
print("RMSE: ", recmetrics.rmse(test.actual, test.cf_predictions))

Recommendations for  American Gods 

1	 Book ID: 233	  Distance: 0.5597519300912784:

2	 Book ID: 60	  Distance: 0.5723053898090453:

3	 Book ID: 67	  Distance: 0.5788764543626139:

4	 Book ID: 148	  Distance: 0.5799694745809161:

5	 Book ID: 704	  Distance: 0.6044421557420616:

6	 Book ID: 733	  Distance: 0.61056149598103:

7	 Book ID: 453	  Distance: 0.6150426503463673:

8	 Book ID: 270	  Distance: 0.6213480740251559:

9	 Book ID: 57	  Distance: 0.6255809547859783:

10	 Book ID: 98	  Distance: 0.6311770656666482:

id: 233 		Book:  El amor en los tiempos del cólera
id: 60 		Book:  The Curious Incident of the Dog in the Night-Time
id: 67 		Book:  A Thousand Splendid Suns
id: 148 		Book:  Girl with a Pearl Earring
id: 704 		Book:  The Cider House Rules
id: 733 		Book:  Snow Falling on Cedars
id: 453 		Book:  Cold Mountain
id: 270 		Book:  Rebecca
id: 57 		Book:  The Secret Life of Bees
id: 98 		Book:  Flickan som lekte med elden
MSE:  0.723485257660798
RMSE:  0.8505793658799853


In [29]:
# Top 10 recommendations for Map of Bones
recommendations_for_MapofBones = get_recomm('Map of Bones', num_neighbors=10, display=True)

for b in recommendations_for_MapofBones[1:]:
    print('id:', b, '\t\tBook: ', get_title(b))

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'book_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()
algo.fit(trainset)

test = algo.test(testset)
test = pd.DataFrame(test)
test.drop("details", inplace=True, axis=1)
test.columns = ['user_id', 'book_id', 'actual', 'cf_predictions']
test.head()
print("MSE: ", recmetrics.mse(test.actual, test.cf_predictions))
print("RMSE: ", recmetrics.rmse(test.actual, test.cf_predictions))

Recommendations for  Map of Bones 

1	 Book ID: 4215	  Distance: 0.16013510276049714:

2	 Book ID: 2239	  Distance: 0.3886720724684398:

3	 Book ID: 1396	  Distance: 0.5642022418554101:

4	 Book ID: 2654	  Distance: 0.7708246054041917:

5	 Book ID: 2416	  Distance: 0.7792669357388019:

6	 Book ID: 2808	  Distance: 0.789958083728866:

7	 Book ID: 3500	  Distance: 0.7995225020245882:

8	 Book ID: 1880	  Distance: 0.8051309253015047:

9	 Book ID: 3096	  Distance: 0.8533399167519998:

10	 Book ID: 2219	  Distance: 0.85405918608189:

id: 4215 		Book:  Time of the Twins
id: 2239 		Book:  Dragons of Spring Dawning
id: 1396 		Book:  Dragons of Autumn Twilight
id: 2654 		Book:  Sojourn (Dark Elf #3)
id: 2416 		Book:  Exile
id: 2808 		Book:  The Crystal Shard
id: 3500 		Book:  Streams of Silver (Icewind Dale #2)
id: 1880 		Book:  Homeland
id: 3096 		Book:  Guardians of the West
id: 2219 		Book:  The Pillars of Creation
MSE:  0.7239266119335638
RMSE:  0.8508387696464964


In [30]:
# Top 10 recommendations for Horror
recommendations_for_Misery = get_recomm('The Magus', num_neighbors=10, display=True)

for b in recommendations_for_Misery[1:]:
    print('id:', b, '\t\tBook: ', get_title(b))

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'book_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()
algo.fit(trainset)

test = algo.test(testset)
test = pd.DataFrame(test)
test.drop("details", inplace=True, axis=1)
test.columns = ['user_id', 'book_id', 'actual', 'cf_predictions']
test.head()
print("MSE: ", recmetrics.mse(test.actual, test.cf_predictions))
print("RMSE: ", recmetrics.rmse(test.actual, test.cf_predictions))

Recommendations for  The Magus 

1	 Book ID: 7048	  Distance: 0.3193071223406899:

2	 Book ID: 7809	  Distance: 0.35798100146220224:

3	 Book ID: 6903	  Distance: 0.38198339975660656:

4	 Book ID: 6377	  Distance: 0.4015261917359346:

5	 Book ID: 4696	  Distance: 0.41082653823693815:

6	 Book ID: 5481	  Distance: 0.436157587512517:

7	 Book ID: 5467	  Distance: 0.43803404354542996:

8	 Book ID: 4825	  Distance: 0.47820606791279674:

9	 Book ID: 6140	  Distance: 0.5008733926997836:

10	 Book ID: 6238	  Distance: 0.6095043791885152:

id: 7048 		Book:  Moscow Rules
id: 7809 		Book:  Prince of Fire
id: 6903 		Book:  A Death In Vienna
id: 6377 		Book:  The Confessor
id: 4696 		Book:  The Defector
id: 5481 		Book:  The English Assassin
id: 5467 		Book:  Portrait of a Spy
id: 4825 		Book:  The Rembrandt Affair
id: 6140 		Book:  The Fallen Angel
id: 6238 		Book:  The Unlikely Spy
MSE:  0.7298857285157758
RMSE:  0.8543334995865349


In [31]:
# Top 10 recommendations for The Kite Runner
recommendations_for_TheKiteRunner = get_recomm('The Kite Runner', num_neighbors=10, display=True)

for b in recommendations_for_TheKiteRunner[1:]:
    print('id:', b, '\t\tBook: ', get_title(b))

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'book_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()
algo.fit(trainset)

test = algo.test(testset)
test = pd.DataFrame(test)
test.drop("details", inplace=True, axis=1)
test.columns = ['user_id', 'book_id', 'actual', 'cf_predictions']
test.head()
print("MSE: ", recmetrics.mse(test.actual, test.cf_predictions))
print("RMSE: ", recmetrics.rmse(test.actual, test.cf_predictions))

Recommendations for  The Kite Runner 

1	 Book ID: 57	  Distance: 0.3873650459835051:

2	 Book ID: 8	  Distance: 0.4321442030047269:

3	 Book ID: 33	  Distance: 0.43465398886887796:

4	 Book ID: 67	  Distance: 0.43533443391040205:

5	 Book ID: 118	  Distance: 0.4359297615395783:

6	 Book ID: 31	  Distance: 0.439949585954313:

7	 Book ID: 16	  Distance: 0.47419409935954904:

8	 Book ID: 46	  Distance: 0.47762647106775946:

9	 Book ID: 22	  Distance: 0.4827477490828982:

10	 Book ID: 43	  Distance: 0.4928997111676672:

id: 57 		Book:  The Secret Life of Bees
id: 8 		Book:  The Catcher in the Rye
id: 33 		Book:  Memoirs of a Geisha
id: 67 		Book:  A Thousand Splendid Suns
id: 118 		Book:  The Joy Luck Club
id: 31 		Book:  The Help
id: 16 		Book:  Män som hatar kvinnor
id: 46 		Book:  Water for Elephants
id: 22 		Book:  The Lovely Bones
id: 43 		Book:  Jane Eyre
MSE:  0.7233911180858377
RMSE:  0.8505240255782536


In [32]:
# Top 10 recommendations for The Adventures of Sherlock Holmes
recommendations_for_TheAdventuresofSherlockHolmes = get_recomm('The Adventures of Sherlock Holmes', num_neighbors=10, display=True)

for b in recommendations_for_TheAdventuresofSherlockHolmes[1:]:
    print('id:', b, '\t\tBook: ', get_title(b))

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'book_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()
algo.fit(trainset)

test = algo.test(testset)
test = pd.DataFrame(test)
test.drop("details", inplace=True, axis=1)
test.columns = ['user_id', 'book_id', 'actual', 'cf_predictions']
test.head()
print("MSE: ", recmetrics.mse(test.actual, test.cf_predictions))
print("RMSE: ", recmetrics.rmse(test.actual, test.cf_predictions))

Recommendations for  The Adventures of Sherlock Holmes 

1	 Book ID: 609	  Distance: 0.709920620460074:

2	 Book ID: 967	  Distance: 0.718126460751016:

3	 Book ID: 865	  Distance: 0.7293485772538959:

4	 Book ID: 911	  Distance: 0.7408548784330786:

5	 Book ID: 1261	  Distance: 0.745052103223284:

6	 Book ID: 1423	  Distance: 0.7497266685129199:

7	 Book ID: 944	  Distance: 0.7499198887949333:

8	 Book ID: 1498	  Distance: 0.7517231887128187:

9	 Book ID: 936	  Distance: 0.756433935157181:

10	 Book ID: 725	  Distance: 0.7575647830097986:

id: 609 		Book:  Cell
id: 967 		Book:  The Dark Half
id: 865 		Book:  Night Shift
id: 911 		Book:  Gerald's Game
id: 1261 		Book:  Eyes of the Dragon
id: 1423 		Book:  Rose Madder
id: 944 		Book:  Desperation
id: 1498 		Book:  Hearts in Atlantis
id: 936 		Book:  Wolves of the Calla
id: 725 		Book:  The Waste Lands
MSE:  0.7217195907481952
RMSE:  0.849540811702531


In [33]:
# Top 10 recommendations for Nights in Rodanthe
recommendations_for_NightsinRodanthe = get_recomm('Nights in Rodanthe', num_neighbors=10, display=True)

for b in recommendations_for_NightsinRodanthe[1:]:
    print('id:', b, '\t\tBook: ', get_title(b))

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'book_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()
algo.fit(trainset)

test = algo.test(testset)
test = pd.DataFrame(test)
test.drop("details", inplace=True, axis=1)
test.columns = ['user_id', 'book_id', 'actual', 'cf_predictions']
test.head()
print("MSE: ", recmetrics.mse(test.actual, test.cf_predictions))
print("RMSE: ", recmetrics.rmse(test.actual, test.cf_predictions))

Recommendations for  Nights in Rodanthe 

1	 Book ID: 676	  Distance: 0.6688698510205567:

2	 Book ID: 938	  Distance: 0.6902854134660964:

3	 Book ID: 904	  Distance: 0.7032737725249492:

4	 Book ID: 589	  Distance: 0.7078718382574001:

5	 Book ID: 1295	  Distance: 0.7104260304290391:

6	 Book ID: 539	  Distance: 0.7491118265668189:

7	 Book ID: 1169	  Distance: 0.7662790149597433:

8	 Book ID: 1143	  Distance: 0.7959584393761469:

9	 Book ID: 1209	  Distance: 0.8087654432189264:

10	 Book ID: 2153	  Distance: 0.8127326887044707:

id: 676 		Book:  Entwined with You
id: 938 		Book:  The Edge of Never
id: 904 		Book:  Gabriel's Inferno
id: 589 		Book:  Easy
id: 1295 		Book:  Never Too Far (Too Far, #2)
id: 539 		Book:  Hopeless
id: 1169 		Book:  Onyx
id: 1143 		Book:  Beautiful Bastard
id: 1209 		Book:  The Marriage Bargain
id: 2153 		Book:  The Secret of Ella and Micha
MSE:  0.7311151310660902
RMSE:  0.8550527066012306
